# Concept Labeling Interface

**Instructions**: 
1. Enter your name in Cell 2
2. Run Cell 3 to load data
3. Run Cell 4 repeatedly to label windows one by one
4. Labels are automatically saved to `labeling_tracker.csv`


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [2]:
# ENTER YOUR NAME HERE
LABELER_NAME = "Your Name"  # Change this!

print(f"Labeler: {LABELER_NAME}")
print("✓ Ready to start labeling")


Labeler: Your Name
✓ Ready to start labeling


In [3]:
# Load data
df_sensor = pd.read_csv('merged_dataset_with_concepts.csv')
df_tracker = pd.read_csv('labeling_tracker.csv')

# Get unlabeled windows for this person
my_windows = df_tracker[(df_tracker['labeler_name'].isna()) | (df_tracker['labeler_name'] == LABELER_NAME)]
unlabeled = my_windows[my_windows['periodicity'].isna()]

print(f"Total windows to label: {len(df_tracker)}")
print(f"Already labeled: {len(df_tracker[df_tracker['periodicity'].notna()])}")
print(f"Your remaining: {len(unlabeled)}")
print(f"\n✓ Data loaded")


Total windows to label: 5
Already labeled: 0
Your remaining: 5

✓ Data loaded


In [4]:
# LABELING CELL - Run this repeatedly to label windows one by one

# Get next unlabeled window
unlabeled = df_tracker[df_tracker['periodicity'].isna()]

if len(unlabeled) == 0:
    print("🎉 ALL WINDOWS LABELED! Great job!")
else:
    current_window = unlabeled.iloc[0]
    window_id = current_window['window_id']
    user_id = current_window['user_id']
    activity = current_window['activity']
    start_time = current_window['start_time']
    end_time = current_window['end_time']
    
    # Extract sensor data
    mask = (df_sensor['user'] == user_id) & \
           (df_sensor['activity'] == activity) & \
           (df_sensor['time_s'] >= start_time - 0.5) & \
           (df_sensor['time_s'] <= end_time + 0.5)
    
    window_data = df_sensor[mask][['time_s', 'x-axis', 'y-axis', 'z-axis']]
    
    # Display window info
    print("="*70)
    print(f"WINDOW {window_id} / {len(df_tracker)}")
    print("="*70)
    print(f"User: {user_id} | Activity: {activity}")
    print(f"Time: {start_time:.2f}s - {end_time:.2f}s")
    print(f"Readings: {len(window_data)} samples")
    print(f"\nRemaining: {len(unlabeled)} windows")
    
    # Visualize
    if len(window_data) > 0:
        fig, axes = plt.subplots(2, 1, figsize=(12, 8))
        
        # All 3 axes together
        ax = axes[0]
        ax.plot(window_data['time_s'], window_data['x-axis'], label='X-axis', linewidth=2, alpha=0.7)
        ax.plot(window_data['time_s'], window_data['y-axis'], label='Y-axis', linewidth=2, alpha=0.7)
        ax.plot(window_data['time_s'], window_data['z-axis'], label='Z-axis', linewidth=2, alpha=0.7)
        ax.set_title(f'Window {window_id}: {activity}', fontweight='bold', fontsize=14)
        ax.set_xlabel('Time (seconds)')
        ax.set_ylabel('Acceleration')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.axvline(start_time, color='green', linestyle='--', alpha=0.5, label='Window start')
        ax.axvline(end_time, color='red', linestyle='--', alpha=0.5, label='Window end')
        
        # Separate axes for clarity
        ax = axes[1]
        time_steps = np.arange(len(window_data))
        ax.plot(time_steps, window_data['x-axis'].values, 'o-', label='X-axis', linewidth=2, markersize=4)
        ax.plot(time_steps, window_data['y-axis'].values, 's-', label='Y-axis', linewidth=2, markersize=4)
        ax.plot(time_steps, window_data['z-axis'].values, '^-', label='Z-axis', linewidth=2, markersize=4)
        ax.set_title('Individual Axis View', fontweight='bold')
        ax.set_xlabel('Sample Index')
        ax.set_ylabel('Acceleration')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Labeling prompts
        print("\n" + "="*70)
        print("ENTER LABELS (refer to labeling_guidelines.md)")
        print("="*70)
        
        print("\n📊 PERIODICITY (0.0, 0.5, or 1.0)")
        print("   1.0 = Regular repeating pattern")
        print("   0.5 = Some pattern but irregular")
        print("   0.0 = No repeating pattern")
        periodicity = float(input("   Enter value: "))
        
        print("\n⏱️  TEMPORAL STABILITY (0.0, 0.5, or 1.0)")
        print("   1.0 = Pattern stable throughout")
        print("   0.5 = Pattern changes once")
        print("   0.0 = Constantly changing")
        temporal = float(input("   Enter value: "))
        
        print("\n🔄 COORDINATION (0.0, 0.5, or 1.0)")
        print("   1.0 = All axes coordinated")
        print("   0.5 = Partially coordinated")
        print("   0.0 = Independent movement")
        coordination = float(input("   Enter value: "))
        
        print("\n💬 CONFIDENCE & NOTES")
        confidence = input("   Confidence (high/medium/low): ")
        notes = input("   Notes (optional, press Enter to skip): ")
        
        # Save to tracker
        idx = df_tracker[df_tracker['window_id'] == window_id].index[0]
        df_tracker.loc[idx, 'periodicity'] = periodicity
        df_tracker.loc[idx, 'temporal_stability'] = temporal
        df_tracker.loc[idx, 'coordination'] = coordination
        df_tracker.loc[idx, 'labeler_name'] = LABELER_NAME
        df_tracker.loc[idx, 'confidence'] = confidence
        df_tracker.loc[idx, 'notes'] = notes
        df_tracker.loc[idx, 'timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        df_tracker.to_csv('labeling_tracker.csv', index=False)
        
        print("\n✅ Label saved!")
        print(f"   Periodicity: {periodicity}")
        print(f"   Temporal Stability: {temporal}")
        print(f"   Coordination: {coordination}")
        print(f"\n▶️  Run this cell again to label the next window")
        
    else:
        print("\n⚠️  No sensor data found for this window")
        print("   Skipping to next window...")
        df_tracker.loc[df_tracker['window_id'] == window_id, 'notes'] = 'No sensor data'
        df_tracker.to_csv('labeling_tracker.csv', index=False)


WINDOW 1 / 5
User: 8 | Activity: Walking
Time: 100.00s - 103.00s
Readings: 0 samples

Remaining: 5 windows

⚠️  No sensor data found for this window
   Skipping to next window...


## Progress Tracking


In [5]:
# Check progress - run anytime
df_tracker = pd.read_csv('labeling_tracker.csv')

print("="*70)
print("LABELING PROGRESS")
print("="*70)

total = len(df_tracker)
labeled = len(df_tracker[df_tracker['periodicity'].notna()])
remaining = total - labeled

print(f"\nOverall: {labeled}/{total} windows ({labeled/total*100:.1f}%)")
print(f"Remaining: {remaining}")

# Per-person progress
print(f"\nPer-person breakdown:")
for labeler in df_tracker['labeler_name'].dropna().unique():
    count = len(df_tracker[df_tracker['labeler_name'] == labeler])
    print(f"  {labeler}: {count} windows")

# Estimate completion
if labeled > 0:
    # Calculate average time per label (if we have timestamps)
    labeled_with_time = df_tracker[df_tracker['timestamp'].notna()]
    if len(labeled_with_time) > 1:
        timestamps = pd.to_datetime(labeled_with_time['timestamp'])
        time_diff = (timestamps.max() - timestamps.min()).total_seconds() / 60  # minutes
        avg_time = time_diff / len(labeled_with_time)
        est_remaining = remaining * avg_time
        print(f"\nEstimated time remaining: {est_remaining:.0f} minutes ({est_remaining/60:.1f} hours)")

# Show confidence distribution
confident = df_tracker[df_tracker['confidence'] == 'high']
uncertain = df_tracker[df_tracker['confidence'].isin(['medium', 'low'])]

print(f"\nConfidence levels:")
print(f"  High: {len(confident)}")
print(f"  Medium/Low: {len(uncertain)} (may need review)")

# Flagged windows
flagged = df_tracker[df_tracker['notes'].notna() & (df_tracker['notes'] != '')]
if len(flagged) > 0:
    print(f"\n⚠️  Flagged windows: {len(flagged)}")
    print("   These need team discussion")


LABELING PROGRESS

Overall: 0/5 windows (0.0%)
Remaining: 5

Per-person breakdown:

Confidence levels:
  High: 0
  Medium/Low: 0 (may need review)

⚠️  Flagged windows: 1
   These need team discussion


## Inter-Rater Reliability Check


In [6]:
# Calculate agreement between labelers
# Run this after calibration session (when multiple people labeled same windows)

from sklearn.metrics import cohen_kappa_score

df_tracker = pd.read_csv('labeling_tracker.csv')

print("="*70)
print("INTER-RATER RELIABILITY")
print("="*70)

# Find windows labeled by multiple people
window_counts = df_tracker.groupby('window_id').size()
overlapping_windows = window_counts[window_counts > 1].index

if len(overlapping_windows) > 0:
    print(f"\nWindows labeled by multiple people: {len(overlapping_windows)}")
    
    # Get all labelers
    labelers = df_tracker['labeler_name'].dropna().unique()
    
    if len(labelers) >= 2:
        # Pairwise comparison
        print(f"\nPairwise Agreement (Cohen's Kappa):")
        print("(Kappa > 0.8 = Excellent, 0.6-0.8 = Good, <0.6 = Needs improvement)\n")
        
        for i, labeler1 in enumerate(labelers):
            for labeler2 in labelers[i+1:]:
                # Get common windows
                df1 = df_tracker[df_tracker['labeler_name'] == labeler1]
                df2 = df_tracker[df_tracker['labeler_name'] == labeler2]
                
                common = pd.merge(df1, df2, on='window_id', suffixes=('_1', '_2'))
                
                if len(common) > 0:
                    print(f"{labeler1} vs {labeler2} ({len(common)} common windows):")
                    
                    # Kappa for each concept
                    kappa_p = cohen_kappa_score(common['periodicity_1'], common['periodicity_2'])
                    kappa_t = cohen_kappa_score(common['temporal_stability_1'], common['temporal_stability_2'])
                    kappa_c = cohen_kappa_score(common['coordination_1'], common['coordination_2'])
                    
                    print(f"  Periodicity: {kappa_p:.3f}")
                    print(f"  Temporal Stability: {kappa_t:.3f}")
                    print(f"  Coordination: {kappa_c:.3f}")
                    print(f"  Average: {(kappa_p + kappa_t + kappa_c)/3:.3f}")
                    print()
                    
                    # Show disagreements
                    disagreements = common[
                        (common['periodicity_1'] != common['periodicity_2']) |
                        (common['temporal_stability_1'] != common['temporal_stability_2']) |
                        (common['coordination_1'] != common['coordination_2'])
                    ]
                    
                    if len(disagreements) > 0:
                        print(f"  Disagreements: {len(disagreements)}/{len(common)} windows")
                        print(f"  Agreement rate: {(1 - len(disagreements)/len(common))*100:.1f}%")
    
    print("\n💡 If Kappa < 0.75:")
    print("   1. Review labeling_guidelines.md together")
    print("   2. Discuss specific disagreements")
    print("   3. Clarify edge cases")
    print("   4. Re-label problematic windows")
    
else:
    print("\n⚠️  No overlapping windows yet.")
    print("   Run calibration session: all 4 people label the same 20-30 windows")


INTER-RATER RELIABILITY

⚠️  No overlapping windows yet.
   Run calibration session: all 4 people label the same 20-30 windows
